In [3]:
import random
from datetime import datetime, timedelta
from tabulate import tabulate

In [ ]:
pip install tabulate

In [4]:
class Car:
    car_types = {
        'COMPACT': {'hourly': 10, 'daily': 50, 'weekly': 125},
        'SDN': {'hourly': 15, 'daily': 75, 'weekly': 150},
        'MUV': {'hourly': 25, 'daily': 100, 'weekly': 150},
        'TRUCKS': {'hourly': 30, 'daily': 120, 'weekly': 200}
    }

    def __init__(self, reg_no, car_type):
        self.reg_no = reg_no
        self.car_type = car_type
        self.available = True
        self.rented_by_customer = None
        self.rented_date = None
        self.return_date = None

    @property
    def rates(self):
        return self.car_types[self.car_type]

In [5]:
class CarInventory:
    def __init__(self):
        self.car_stock = {}

    def generate_car_stock(self, num_cars):
        for _ in range(num_cars):
            reg_no   = random.randint(10000, 50000)
            car_type = random.choice(list(Car.car_types.keys()))
            car      = Car(reg_no, car_type)
            
            self.car_stock[reg_no] = car
            
        return f"{num_cars} cars generated successfully!"
    
    def display_stock(self):
        car_type_count = {}
        total_stock = {}

        for car in self.car_stock.values():
            car_type = car.car_type
            if car_type in total_stock:
                total_stock[car_type] += 1
            else:
                total_stock[car_type] = 1

            if car.available:
                if car_type in car_type_count:
                    car_type_count[car_type] += 1
                else:
                    car_type_count[car_type] = 1

        if not car_type_count:
            return "No cars available"

        available_cars = []
        for car_type, count in car_type_count.items():
            total = total_stock[car_type]
            rates = Car.car_types[car_type]
            rental_rates = f"Hourly: ${rates['hourly']}, Daily: ${rates['daily']}, Weekly: ${rates['weekly']}"
            available_cars.append((car_type, total, count, rental_rates))

        headers = ["Car Type", "Total Stock", "Number of Available Cars", "Rental Rates"]
        table = tabulate(available_cars, headers, tablefmt="pretty")
        return table

In [6]:
class Booking:
    def __init__(self, customer_name, car, rental_basis):
        self.customer_name = customer_name
        self.car = car
        self.rental_basis = rental_basis
        self.rental_date = datetime.now()
        self.return_date = None

In [7]:
class BookingManager:
    def __init__(self):
        self.customer_bookings = {}

    def book_car(self, car_inventory, customer_name, car_type, qty, rental_basis):
        if car_type not in Car.car_types:
            return "Invalid car type entered."

        if rental_basis not in ['hourly', 'daily', 'weekly']:
            return "Invalid rental basis entered."

        if qty <= 0:
            return "Quantity must be a positive integer."

        available_cars = [car for car in car_inventory.car_stock.values() if car.available and car.car_type == car_type]

        if not available_cars:
            return f"No available cars of {car_type} type."

        if qty > len(available_cars):
            return f"Not enough {car_type} cars available. Available: {len(available_cars)}"

        booking_ref = f"{customer_name}_{car_type}_{rental_basis}_{datetime.now().strftime('%Y%m%d%H%M%S%f')}"

        booking_details = []
        for car in random.sample(available_cars, qty):
            car.available = False
            car.rented_by_customer = customer_name
            car.rented_date = datetime.now()
            car.return_date = None
            booking = Booking(customer_name, car, rental_basis)
            booking_details.append(booking)

        self.customer_bookings[booking_ref] = booking_details

        return f"Booking Successful! Booking Reference: {booking_ref}"

    def display_customer_bookings(self):
        booking_data = []
        headers = ["Booking Ref", "Customer Name", "Car Type", "Reg No", "Rental Basis", "Rental Date", "Return Date"]
        for booking_ref, bookings in self.customer_bookings.items():
            for booking in bookings:
                booking_data.append([
                    booking_ref,
                    booking.customer_name,
                    booking.car.car_type,
                    booking.car.reg_no,
                    booking.rental_basis,
                    booking.rental_date.strftime('%Y-%m-%d %H:%M:%S'),
                    booking.return_date
                ])
        table = tabulate(booking_data, headers, tablefmt="pretty")
        return table

    def return_car(self, car_inventory, customer_name, reg_no):
        active_bookings = []
        for booking_ref, bookings in self.customer_bookings.items():
            for booking in bookings:
                if booking.customer_name == customer_name and booking.return_date is None:
                    active_bookings.append((booking_ref, booking))

        if not active_bookings:
            return "No active bookings found for the customer."

        booking_details = None
        for booking_ref, booking in active_bookings:
            if booking.car.reg_no == reg_no:
                booking_details = booking
                break

        if not booking_details:
            return "Invalid registration number or the car is not currently rented by the customer."

        rental_date = booking_details.rental_date
        return_date = datetime.now()
        rental_basis = booking_details.rental_basis
        car_type = booking_details.car.car_type
        rental_rate = Car.car_types[car_type][rental_basis]

        rental_duration = return_date - rental_date
        if rental_basis == 'hourly':
            rental_hours = rental_duration.total_seconds() / 3600
            bill_amount = rental_hours * rental_rate
        elif rental_basis == 'daily':
            rental_days = rental_duration.days
            bill_amount = rental_days * rental_rate
        elif rental_basis == 'weekly':
            rental_weeks = rental_duration.days / 7
            bill_amount = rental_weeks * rental_rate

        bill_amount = round(bill_amount, 2)
        bill_paid = input("Is the bill paid? (yes/no): ").strip().lower()

        if bill_paid == 'yes':
            booking_details.return_date = return_date
            car_inventory.car_stock[reg_no].available = True
            car_inventory.car_stock[reg_no].rented_by_customer = None
            car_inventory.car_stock[reg_no].rented_date = None
            car_inventory.car_stock[reg_no].return_date = None
            return f"Car returned successfully. Bill of ${bill_amount} has been paid."
        else:
            return "Car return not completed. Bill not paid."

In [8]:
car_inventory = CarInventory()

In [9]:
car_inventory.car_stock

{}

In [10]:
car_inventory.display_stock()

'No cars available'

In [11]:
car_inventory.generate_car_stock(20)

'20 cars generated successfully!'

In [12]:
print(car_inventory.display_stock())

+----------+-------------+--------------------------+----------------------------------------+
| Car Type | Total Stock | Number of Available Cars |              Rental Rates              |
+----------+-------------+--------------------------+----------------------------------------+
|  TRUCKS  |      4      |            4             | Hourly: $30, Daily: $120, Weekly: $200 |
|   SDN    |      5      |            5             | Hourly: $15, Daily: $75, Weekly: $150  |
|   MUV    |      5      |            5             | Hourly: $25, Daily: $100, Weekly: $150 |
| COMPACT  |      6      |            6             | Hourly: $10, Daily: $50, Weekly: $125  |
+----------+-------------+--------------------------+----------------------------------------+


In [17]:
booking_manager = BookingManager()

In [18]:
booking_manager.display_customer_bookings()

'+-------------+---------------+----------+--------+--------------+-------------+-------------+\n| Booking Ref | Customer Name | Car Type | Reg No | Rental Basis | Rental Date | Return Date |\n+-------------+---------------+----------+--------+--------------+-------------+-------------+\n+-------------+---------------+----------+--------+--------------+-------------+-------------+'

In [19]:
while True:
    print("\nCar Rental Management System")
    print("1. Generate stock of cars")
    print("2. Display the car stock")
    print("3. Rent car(s)")
    print("4. Return car")
    print("5. Exit")
    choice = input("Enter your choice: ")

    if choice == '1':
        num_cars = int(input("Enter the number of cars to generate: "))
        result = car_inventory.generate_car_stock(num_cars)
        print(result)

    elif choice == '2':
        print(car_inventory.display_stock())

    elif choice == '3':
        customer_name = input("Enter customer name: ")
        car_type = input("Enter car type (COMPACT, SDN, MUV, TRUCKS): ").upper()
        qty = int(input("Enter quantity: "))
        rental_basis = input("Enter rental basis (hourly, daily, weekly): ").lower()
        result = booking_manager.book_car(car_inventory, customer_name, car_type, qty, rental_basis)
        print(result)

    elif choice == '4':
        customer_name = input("Enter customer name: ")
        print(booking_manager.display_customer_bookings())
        
        reg_no = int(input("Enter the registration number of the car being returned: "))
        result = booking_manager.return_car(car_inventory, customer_name, reg_no)
        print(result)

    elif choice == '5':
        print("Exiting the system. Goodbye!")
        break

    else:
        print("Invalid choice. Please try again.")


Car Rental Management System
1. Generate stock of cars
2. Display the car stock
3. Rent car(s)
4. Return car
5. Exit


Enter your choice:  4
Enter customer name:  bks


+-------------+---------------+----------+--------+--------------+-------------+-------------+
| Booking Ref | Customer Name | Car Type | Reg No | Rental Basis | Rental Date | Return Date |
+-------------+---------------+----------+--------+--------------+-------------+-------------+
+-------------+---------------+----------+--------+--------------+-------------+-------------+


Enter the registration number of the car being returned:  5


No active bookings found for the customer.

Car Rental Management System
1. Generate stock of cars
2. Display the car stock
3. Rent car(s)
4. Return car
5. Exit


...

Car returned successfully. Bill of $0.0 has been paid.

Car Rental Management System
1. Generate stock of cars
2. Display the car stock
3. Rent car(s)
4. Return car
5. Exit


Enter your choice:  2


+----------+-------------+--------------------------+----------------------------------------+
| Car Type | Total Stock | Number of Available Cars |              Rental Rates              |
+----------+-------------+--------------------------+----------------------------------------+
|  TRUCKS  |     17      |            17            | Hourly: $30, Daily: $120, Weekly: $200 |
|   SDN    |     16      |            15            | Hourly: $15, Daily: $75, Weekly: $150  |
|   MUV    |     13      |            11            | Hourly: $25, Daily: $100, Weekly: $150 |
| COMPACT  |     14      |            14            | Hourly: $10, Daily: $50, Weekly: $125  |
+----------+-------------+--------------------------+----------------------------------------+

Car Rental Management System
1. Generate stock of cars
2. Display the car stock
3. Rent car(s)
4. Return car
5. Exit


Enter your choice:  5


Exiting the system. Goodbye!
